In [1]:

import math
import numpy as np


In [2]:
import csv
maxage = 0;
minage = 2030
with open('train2016.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    noTrainingExamples = 5568 # number of items in csv...  
    noColumns = 108 # how many columns will we store... 
    line_count = 0
    outputs = np.zeros(noTrainingExamples)
    matrices = np.zeros((noTrainingExamples, noColumns))
    columnNames = np.empty([1,noColumns], dtype=object)
    userIDs = np.empty([1,noTrainingExamples])
    for row in csv_reader:
            
        matrix = np.zeros(noColumns)
#             matrix[0] = float(row['YOB'])
        userIDs[0,line_count] = row['USER_ID']
        # YOB
        columnNames[0,0] = 'YOB'
        try:
            matrix[0] = (((float(row['YOB'])%1900)/100))
             #matrix[0] = (((float(row['YOB'])%1900)/100)-0.5)
            if matrix[0] > maxage:
                maxage = matrix[0]
            if matrix[0] < minage:
                minage = matrix[0]
        except ValueError:
            matrix[0] = -1
        
        # GENDER
        columnNames[0,1] = 'Gender'
        if 'Male' in row['Gender']:
            matrix[1] = 1
        elif 'Female' in row['Gender']:
            matrix[1] = 0
        else:
            matrix[1] = -1
        
            
        #INCOME
        columnNames[0,2] = 'Income'
        if 'over $150,000' in row['Income']:
            matrix[2] = 1
        elif '$100,001 - $150,000' in row['Income']:
            matrix[2] = 0.8
        elif '$75,000 - $100,000' in row['Income']:
            matrix[2] = 0.6
        elif '$50,000 - $74,999' in row['Income']:
            matrix[2] = 0.4
        elif '$25,001 - $50,000' in row['Income']:
            matrix[2] = 0.1
        elif 'under $25,000' in row['Income']:
            matrix[2] = 0
        else:
            matrix[2] = -1
        
        # HOUSEHOLD
        columnNames[0,3] = 'HouseholdStatus'
        if 'Married' in row['HouseholdStatus']:
            matrix[3] = 1
        elif 'Domestic' in row['HouseholdStatus']:
            matrix[3] = 0.5
        elif 'Single' in row['HouseholdStatus']:
            matrix[3] = 0
        else:
            matrix[3] = -1
        
        # HAS KIDS OR NOT
        columnNames[0,4] = 'Kids'
        if 'w/' in row['HouseholdStatus']:
            matrix[4] = 1
        elif 'no' in row['HouseholdStatus']:
            matrix[4] = 0
        else:
            matrix[4] = -1
            
        
        # EDUCATION
        columnNames[0,5] = 'EducationLevel'
        if 'Doctoral' in row['EducationLevel']:
            matrix[5] = 1
        elif 'Master' in row['EducationLevel']:
            matrix[5] = 0.9
        elif 'Bachelor' in row['EducationLevel']:
            matrix[5] = 0.7
        elif 'Undergraduate' in row['EducationLevel']:
            matrix[5] = 0.6
        elif 'Associate' in row['EducationLevel']:
            matrix[5] = 0.5
        elif 'High School Diploma' in row['EducationLevel']:
            matrix[5] = 0.25
        elif 'K-12' in row['EducationLevel']:
            matrix[5] = 0.0
        else:
            matrix[5] = -1
        
        
        
            
            
#         print(row['YOB'])
#         if "w/kids" in row['HouseholdStatus']:
#             print("HEY!!!")

        nextIndex = 6
        for col in row:
            if col in ['USER_ID', 'YOB', 'Gender', 'Income', 'HouseholdStatus', 'EducationLevel','Party']:
                continue
            if row[col] in ['Yes', 'Check!', 'Optimist', 'Mom', 'Rent', 'Yay people!', 'Online', 'Yes!', 'Socialize', 'Cautious', 'Mac', 'Supportive', 'Tunes', 'People', 'TMI', 'Start', 'Circumstances', 'A.M.', 'Happy', 'Hot headed', 'Standard hours', 'Idealist', 'Giving', 'Study first', 'Science', 'Public']:
                matrix[nextIndex] = 1
            elif row[col] in ['No', 'Only-child', 'Nope', 'Pessimist', 'Dad', 'Own', 'Grrr people', 'In-person', 'Umm...', 'Space', 'Risk-friendly', 'PC', 'Demanding', 'Talk', 'Technology', 'Mysterious', 'End', 'Me', 'P.M.', 'Right', 'Cool headed', 'Odd hours', 'Pragmatist', 'Receiving', 'Try first', 'Art', 'Private']:
                matrix[nextIndex] = 0
            else:
                matrix[nextIndex] = -1
            columnNames[0,nextIndex] = col
            nextIndex += 1
            
        try:
            partyAff = row['Party']
            columnNames[0,nextIndex] = 'Party'
            if 'Dem' in row['Party']:
                matrix[nextIndex] = 1
                outputs[line_count] = 1
            elif 'Rep' in row['Party']:
                matrix[nextIndex] = 0
                outputs[line_count] = 0
            else:
                matrix[nextIndex] = -1
                outputs[line_count] = -1
        except:
            matrix[nextIndex] = -1
        
        
    
        matrices[line_count,:] = matrix
        line_count +=1
        if line_count >= noTrainingExamples:
            break
        
    


In [3]:
maxage
minage

0.0

In [4]:
print(len(columnNames[0,:]))

108


In [5]:
import pandas as pd
data = pd.DataFrame(data=matrices,
                   columns=columnNames[0,:])
print(data.head())

    YOB  Gender  Income  HouseholdStatus  Kids  EducationLevel  Q124742  \
0  0.38     1.0    -1.0              1.0   1.0           -1.00      0.0   
1  0.70     0.0     1.0              0.5   1.0            0.70     -1.0   
2  0.97     1.0     0.6              0.0   0.0            0.25     -1.0   
3  0.83     1.0     0.8              1.0   1.0            0.70      0.0   
4  0.84     0.0     0.4              1.0   1.0            0.25      0.0   

   Q124122  Q123464  Q123621  ...    Q99716  Q99581  Q99480  Q98869  Q98578  \
0     -1.0      0.0      0.0  ...       0.0     0.0    -1.0     0.0    -1.0   
1      1.0      0.0      0.0  ...      -1.0    -1.0     0.0     0.0     0.0   
2      1.0      1.0      0.0  ...       0.0     0.0     0.0     1.0     0.0   
3      1.0      0.0      1.0  ...       0.0     0.0     1.0     1.0     0.0   
4      1.0      0.0      0.0  ...       0.0     0.0     1.0     0.0     0.0   

   Q98059  Q98078  Q98197  Q96024  Party  
0     0.0     0.0     0.0     1

In [6]:
min(data['YOB'])

-1.0

In [7]:
# import matplotlib.pyplot as plt
# # import tensorflow as tf
# %matplotlib inline
# # import os
# # os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu

In [8]:
print(data.shape)

(5568, 108)


In [9]:
X = data.drop('Party', axis=1)
y = data['Party']
print(X.head())
print(y.head())

    YOB  Gender  Income  HouseholdStatus  Kids  EducationLevel  Q124742  \
0  0.38     1.0    -1.0              1.0   1.0           -1.00      0.0   
1  0.70     0.0     1.0              0.5   1.0            0.70     -1.0   
2  0.97     1.0     0.6              0.0   0.0            0.25     -1.0   
3  0.83     1.0     0.8              1.0   1.0            0.70      0.0   
4  0.84     0.0     0.4              1.0   1.0            0.25      0.0   

   Q124122  Q123464  Q123621   ...    Q100010  Q99716  Q99581  Q99480  Q98869  \
0     -1.0      0.0      0.0   ...        1.0     0.0     0.0    -1.0     0.0   
1      1.0      0.0      0.0   ...       -1.0    -1.0    -1.0     0.0     0.0   
2      1.0      1.0      0.0   ...        1.0     0.0     0.0     0.0     1.0   
3      1.0      0.0      1.0   ...        0.0     0.0     0.0     1.0     1.0   
4      1.0      0.0      0.0   ...        1.0     0.0     0.0     1.0     0.0   

   Q98578  Q98059  Q98078  Q98197  Q96024  
0    -1.0     0.0 

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
# import numpy as np
import os
# import pandas as pd
import re
import seaborn as sns

W0322 14:43:00.251849 4567782848 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [11]:

msk = np.random.rand(len(data)) < 0.8
train_data = data[msk]
test_data = data[~msk]
print(len(train_data))
print(len(test_data))


# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_data, train_data["Party"], num_epochs=None, shuffle=True)

# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_data, train_data["Party"], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    test_data, test_data["Party"], shuffle=False)

4435
1133


In [12]:
4429+1139

5568

In [13]:
X = data.drop('Party', axis=1)
y = data['Party']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20)

In [14]:
#50 10 50
feature_columns = [tf.feature_column.numeric_column(key = key) for key in X_train.columns]
estimator = tf.estimator.DNNClassifier(
                hidden_units=[50,50,25],
                feature_columns=feature_columns,
                n_classes=2,
                optimizer=tf.train.AdagradOptimizer(learning_rate=0.003))

INFO:tensorflow:Using default config.


I0322 14:43:01.436122 4567782848 estimator.py:1739] Using default config.


W0322 14:43:01.440447 4567782848 estimator.py:1760] Using temporary folder as model directory: /var/folders/y9/7kjb4fj50lngbqcff3_4y4n40000gn/T/tmpo9a9loyx


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/y9/7kjb4fj50lngbqcff3_4y4n40000gn/T/tmpo9a9loyx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a32ede4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0322 14:43:01.441962 4567782848 estimator.py:201] Using config: {'_model_dir': '/var/folders/y9/7kjb4fj50lngbqcff3_4y4n40000gn/T/tmpo9a9loyx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a32ede4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
print(estimator)

In [16]:
estimator.train(input_fn=train_input_fn, steps=1000)

Instructions for updating:
Colocations handled automatically by placer.


W0322 14:43:01.472015 4567782848 deprecation.py:323] From /Users/suhwanchoi/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


W0322 14:43:01.594262 4567782848 deprecation.py:323] From /Users/suhwanchoi/anaconda3/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


...

I0322 14:43:17.306802 4567782848 basic_session_run_hooks.py:247] loss = 73.772156, step = 901 (1.476 sec)


INFO:tensorflow:Saving checkpoints for 1000 into /var/folders/y9/7kjb4fj50lngbqcff3_4y4n40000gn/T/tmpo9a9loyx/model.ckpt.


I0322 14:43:19.310253 4567782848 basic_session_run_hooks.py:594] Saving checkpoints for 1000 into /var/folders/y9/7kjb4fj50lngbqcff3_4y4n40000gn/T/tmpo9a9loyx/model.ckpt.


INFO:tensorflow:Loss for final step: 78.25363.


I0322 14:43:19.747477 4567782848 estimator.py:359] Loss for final step: 78.25363.


In [17]:
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

print("Training set accuracy: {accuracy}".format(**train_eval_result))
print("Test set accuracy: {accuracy}".format(**test_eval_result))

INFO:tensorflow:Calling model_fn.


I0322 14:43:19.870528 4567782848 estimator.py:1111] Calling model_fn.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


W0322 14:43:21.289723 4567782848 deprecation.py:323] From /Users/suhwanchoi/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/metrics_impl.py:2002: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


W0322 14:43:21.774491 4567782848 metrics_impl.py:783] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.


...

INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.5789938, accuracy_baseline = 0.51632833, auc = 0.6340695, auc_precision_recall = 0.6446816, average_loss = 0.6643815, global_step = 1000, label/mean = 0.51632833, loss = 83.63825, precision = 0.58231705, prediction/mean = 0.5302972, recall = 0.6529915


I0322 14:43:27.712585 4567782848 estimator.py:1979] Saving dict for global step 1000: accuracy = 0.5789938, accuracy_baseline = 0.51632833, auc = 0.6340695, auc_precision_recall = 0.6446816, average_loss = 0.6643815, global_step = 1000, label/mean = 0.51632833, loss = 83.63825, precision = 0.58231705, prediction/mean = 0.5302972, recall = 0.6529915


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: /var/folders/y9/7kjb4fj50lngbqcff3_4y4n40000gn/T/tmpo9a9loyx/model.ckpt-1000


I0322 14:43:27.718848 4567782848 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1000: /var/folders/y9/7kjb4fj50lngbqcff3_4y4n40000gn/T/tmpo9a9loyx/model.ckpt-1000


Training set accuracy: 0.6755355000495911
Test set accuracy: 0.5789937973022461


In [18]:
# https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub

SyntaxError: invalid syntax (<ipython-input-19-af826e2d1b66>, line 1)